# Locality Sensitive Hashing (LSH)
Locality-Sensitive Hashing (LSH) is a technique used to solve the approximate or exact near-neighbour search problem in high-dimensional spaces. The basic idea behind LSH is to hash similar items to the same “buckets” with high probability, while hashing dissimilar items to different buckets with high probability. This allows for efficient search of large datasets, as items that are similar will be found in the same bucket and can be compared more quickly than items that are dissimilar.

LSH has many applications, including nearest neighbour search, clustering, image and text retrieval, and recommendation systems. It has been successfully applied to problems in computer vision, information retrieval, data mining, and machine learning.

- Retrieval of nearest neighbours of a query point q using LSH works as follows:
    - Hash all data points using locality-sensitive hash
    - Compute locality-sensitive hash of query point
    - All data points in the bucket are candidate near neighbours
    - Compute distances to candidate points to find true nearest neighbours
- Locality-sensitive hashing can be used with minhash
signatures:
• Divide signature matrix into b bands consisting of r rows each
• Hash each sub-signature of length r into a hash table per band
• Two sets with at least one identical sub-signature will hash in
the same bucket (at least once)
    → Candidate column pairs for similarity

### Normal vs. LSH

- Normal hash functions try to minimize the probability of collision
- LSH hash functions try to maximize probability of similar items colliding

### Problem with LSH Projections

![Collision and Split](../assets/collision.png)

Collision and Split

********AND-Construction →******** Using multiple projections in an LSH resolves “collisions”. Points are candidates if they occur in all query bins

****************************OR-Construction →****************************  Using multiple separate hash tables when doing LSH resolves “splits”. Points are candidate neighbours if candidate in any of the hash tables

### LSH in Space Summary

- Hashes are done using random projections
- Combining projections using AND reduces FP and slightly increases FN
- Combining sets of projections using OR reduces FN and slightly increases FP
- Cascading AND/OR constructions for optimal performance

### Banding

Banding is a technique used in Locality-Sensitive Hashing (LSH) to improve the efficiency of the hashing process. It involves dividing the signature matrix into "bands," which consist of a certain number of rows. Each band is then hashed separately, and the resulting hash values are combined to generate an overall hash value for the signature. This technique can help to reduce the number of false positives in the LSH process, improving the accuracy of the results.

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import random

# a should be similar to b
a = "This is an amazing test string that tests similarity"
b = "This is another amazing test string that also tests similarity"
c = "I am completly unrelated"

dataset = [
  a, b, c
]

## Shingles

In [2]:
def shingle(text: str, k: int):
  return {text[i:i+k] for i in range(len(text)-k+1)}

In [3]:
import functools

vocabulary = list(functools.reduce(lambda a, b: a.union(b), [shingle(i, 2) for i in dataset]))

In [4]:
def one_hot(vocabulary, shingles):
  return [1 if i in shingles else 0 for i in vocabulary]

In [5]:
transformed_dataset = [one_hot(vocabulary, shingle(i, 2)) for i in dataset]

## Signatures

Signatures are short integer vectors that represent the sets, and reflect their similarity

## MinWise Hashing

Minwise hashing is a technique used in Locality-Sensitive Hashing to estimate the similarity between two sets. It works by representing a set as a series of hash values, and then comparing the hash values of two sets to estimate their similarity.

### Algorithm

1. Initialize a set of hash functions H.
2. For each element in the set, compute the minimum hash value for each hash function in H.
3. The resulting set of minimum hash values is the signature of the set.

### Example

Suppose we have two sets, A = {1, 2, 3, 4} and B = {2, 3, 5, 7}. We can use minwise hashing to estimate their similarity as follows:

1. Choose two hash functions, h1(x) = (3x + 1) mod 5 and h2(x) = (5x + 2) mod 7.
2. Compute the minimum hash values for each set:

    A: h1(1) = 4, h2(1) = 3
    h1(2) = 1, h2(2) = 0
    h1(3) = 4, h2(3) = 5
    h1(4) = 2, h2(4) = 0
    Signature(A) = {1, 0, 4, 0}

    B: h1(2) = 1, h2(2) = 0
    h1(3) = 4, h2(3) = 5
    h1(5) = 0, h2(5) = 0
    h1(7) = 3, h2(7) = 3
    Signature(B) = {0, 0, 4, 0}

3. The Jaccard similarity between A and B can be estimated as the fraction of components in their signatures that are equal:

    Jaccard(A, B) ≈ 2/4 = 0.5



In [6]:
import random

class MinHash:
  def __init__(self, size, hash_functions_count: int):
    self.permutations = [list(range(size)) for i in range(hash_functions_count)]
    for i in self.permutations:
      random.shuffle(i)

  def get_signature(self, X):
    sign = []
    for permutation in self.permutations:
      for i in permutation:
        if X[i] == 1:
          sign.append(i)
          break;
    return sign

minhasher = MinHash(len(vocabulary), 20)
signatures = [minhasher.get_signature(i) for i in transformed_dataset]

In [7]:
def jaccard_similarity(A: set, B: set) -> float:
  union = A.union(B)
  intersection = A.intersection(B)
  return len(intersection)/len(union)

In [8]:
jaccard_similarity(set(signatures[0]), set(signatures[1]))

0.6

In [9]:
jaccard_similarity(set(signatures[1]), set(signatures[2]))

0.037037037037037035

## Locality Sensitive Hashing Implementation

In [10]:
def split_signature(signature, b):
  assert len(signature) % b == 0
  return [signature[i:i+b] for i in range(0, len(signature), b)]
split_signature(signatures[1],4)

[[17, 40, 52, 12],
 [20, 20, 13, 23],
 [54, 49, 52, 57],
 [15, 49, 51, 36],
 [38, 28, 11, 12]]

In [11]:
def are_candidate_pairs(signA, signB, bands):
  splitA = split_signature(signA, bands)
  splitB = split_signature(signB, bands)

  return any([a==b for a,b in zip(splitA, splitB)])
print("0 with 1 are pairs? ", are_candidate_pairs(signatures[0],signatures[1],4))
print("1 with 2 are pairs? ", are_candidate_pairs(signatures[1],signatures[2],4))
print("0 with 2 are pairs? ", are_candidate_pairs(signatures[0],signatures[2],4))

0 with 1 are pairs?  True
1 with 2 are pairs?  False
0 with 2 are pairs?  False
